In [1]:
if False:
    from google.colab import drive
    drive.mount('/content/drive')

#### 1. Preprocessing images

For images, it is necessary, at least as the bare minimum, to all have the same size. If the model underperforms, more pre processing steps are to be added 

In [ ]:
import os
import numpy as np
from PIL import Image
from tqdm.auto import tqdm

home = "C:\\Users\\αααοοο\\galaxy-zoo-the-galaxy-challenge\\"


train_folder_img = home + 'images_training_rev1\\images_training_rev1\\'
test_folder_img = home + 'images_test_rev1\\images_test_rev1\\'

train_folder_proc = home + 'images_training_fin\\'
test_folder_proc = home + 'images_test_fin\\'

preprocessing = False #toggle this to enable preprocessing
width = 227 #typical for architectures like 

if preprocessing:
    for name in tqdm(os.listdir(train_folder_img), desc="Loading Train Files"): #managing a lot of files. added progress bars for tracking time
        img_path = train_folder_img + name

        with Image.open(img_path) as img:
            w, h = img.size
            height = round(h * (width / w))
            resized = img.resize((width, height), Image.Resampling.LANCZOS) #resampling, suggested by the library
            resized.save(train_folder_proc + name)
    
    for name in tqdm(os.listdir(test_folder_img), desc="Loading Test Files"):
        img_path = test_folder_img + name

        with Image.open(img_path) as img:
            w, h = img.size
            height = round(h * (width / w))
            resized = img.resize((width, height), Image.Resampling.LANCZOS)
            resized.save(test_folder_proc + name)
        
    

Making a mendatory check that every image is 227x227. This part is also necessary for creating the datasets.

In [4]:
test_image_names = []
train_image_names = []

for name in tqdm(os.listdir(train_folder_proc), desc="Validating Train Files"): #managing a lot of files. added progress bars for tracking time
    img_path = train_folder_proc + name
    train_image_names.append(name)

    with Image.open(img_path) as img:
        w, h = img.size
        if w != 227 or h != 227:
            print ("Not matching size")
            break

# #comment this for fast checks
# for name in tqdm(os.listdir(test_folder_proc), desc="Validating Test Files"):
#     img_path = test_folder_proc + name
#     test_image_names.append(name)

#     with Image.open(img_path) as img:
#         w, h = img.size
#         if w != 227 or h != 227:
#             print ("Not matching size")
#             break

Validating Train Files:   0%|          | 0/61578 [00:00<?, ?it/s]

#### 2. Loading labels 

In [5]:
import pandas as pd

train_solutions = home + 'training_solutions_rev1\\training_solutions_rev1.csv'

train_sols = pd.read_csv(train_solutions)
train_sols.head()

,GalaxyID,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,...,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
0,100008,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,...,0.000000,0.279952,0.138445,0.000000,0.000000,0.092886,0.0,0.0,0.0,0.325512
1,100023,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,...,0.018764,0.000000,0.131378,0.459950,0.000000,0.591328,0.0,0.0,0.0,0.000000
2,100053,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,100078,0.693377,0.238564,0.068059,0.000000,0.238564,0.109493,0.129071,0.189098,0.049466,...,0.000000,0.094549,0.000000,0.094549,0.189098,0.000000,0.0,0.0,0.0,0.000000
4,100090,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000


In [6]:
#again, sanity check, seeing whether all labels are here
train_sols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61578 entries, 0 to 61577
Data columns (total 38 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   GalaxyID   61578 non-null  int64  
 1   Class1.1   61578 non-null  float64
 2   Class1.2   61578 non-null  float64
 3   Class1.3   61578 non-null  float64
 4   Class2.1   61578 non-null  float64
 5   Class2.2   61578 non-null  float64
 6   Class3.1   61578 non-null  float64
 7   Class3.2   61578 non-null  float64
 8   Class4.1   61578 non-null  float64
 9   Class4.2   61578 non-null  float64
 10  Class5.1   61578 non-null  float64
 11  Class5.2   61578 non-null  float64
 12  Class5.3   61578 non-null  float64
 13  Class5.4   61578 non-null  float64
 14  Class6.1   61578 non-null  float64
 15  Class6.2   61578 non-null  float64
 16  Class7.1   61578 non-null  float64
 17  Class7.2   61578 non-null  float64
 18  Class7.3   61578 non-null  float64
 19  Class8.1   61578 non-null  float64
 20  Class8

In [7]:
#the number of labels matches the number of training examples. It is also important to remove the Galaxy index coumn
train_sols = train_sols.drop("GalaxyID", axis=1)
train_sols.head()

,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,Class5.1,...,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
0,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,0.0,...,0.000000,0.279952,0.138445,0.000000,0.000000,0.092886,0.0,0.0,0.0,0.325512
1,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,0.0,...,0.018764,0.000000,0.131378,0.459950,0.000000,0.591328,0.0,0.0,0.0,0.000000
2,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,0.693377,0.238564,0.068059,0.000000,0.238564,0.109493,0.129071,0.189098,0.049466,0.0,...,0.000000,0.094549,0.000000,0.094549,0.189098,0.000000,0.0,0.0,0.0,0.000000
4,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000


#### 3. Create dataloaders

A problem encountered here is, we cannot just load tens of thousands of images on RAM (neither on CPU nor GPU memory). Among the possible solutions, it was decided to create a dataloader that yields batches of training examples.

In [8]:
import tensorflow as tf

class GalaxyDataset:
    def __init__(self, images_path, images_names, labels, batch_size = 512):
        self.images_path = images_path
        self.images_names = images_names
        self.batch_size = batch_size
        self.labels = labels

    def generator(self):
        image_batch = []
        labels_batch = []
        counter = 0
        
        while True:
            if counter == len(self.images_names):
                break

            image_path = self.images_path + self.images_names[counter]
            with Image.open(image_path) as img:
                img = img.convert("RGB")
                image_batch.append(np.array(img).astype(np.float32))
            
            labels_batch.append(self.labels[counter])

            counter = counter + 1

            if (counter%self.batch_size == 0) or (counter == len(self.images_names)):
                print (counter)
                X = np.stack(image_batch, axis=0).astype(np.float32)
                y = np.array(labels_batch).astype(np.float32)

                image_batch = []
                labels_batch = []

                yield X, y 
    
    def get_tf_dataset(self):
        output_signature = (
            tf.TensorSpec(shape=(None, 227, 227, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, 37), dtype=tf.float32)
        )

        dataset = tf.data.Dataset.from_generator(
                    self.generator,
                    output_signature=output_signature
                )

        return dataset

In [9]:
labels = train_sols.to_numpy()

Picking last 20% as validation

In [10]:
total_n = len(train_image_names)
validation_n = int(round(total_n*0.2))

val_names  = train_image_names[:validation_n]
train_names = train_image_names[validation_n:]

val_labels  = labels[:validation_n]
train_labels = labels[validation_n:]

In [11]:
train_ds = GalaxyDataset(train_folder_proc, train_names, train_labels)
val_ds = GalaxyDataset(train_folder_proc, val_names, val_labels)

tf_train_ds = train_ds.get_tf_dataset()
tf_val_ds = val_ds.get_tf_dataset()

#sanity checking the dataloader's dimensions
# for batch_i, (X, y) in enumerate(tf_train_ds):
#     print(batch_i, X.shape, y.shape)
    
# for batch_i, (X, y) in enumerate(tf_val_ds):
#     print(batch_i, X.shape, y.shape)

#### 4. Defining the architecture

As a starting step, we took a vanilla implementation of AlexNet and tried to train it for the purposes of this project. AlexNet was selected due to its reputation as a well performing model in image classification tasks (at least for its time). Although our task is not really image classification, it is still interesting to see its performance for this task.

2 important things before the architecture is define are:  
- Normal AlexNet is made for image classification. That means that its normal implementation involves softmax as the activation function of the last dense layer. For our task, we elected to use signmoid as the activation function of the last layer, since the target values are continuous (from 0 to 1).
- AlexNet does not really contain any method of generalization, other that adding dropout between the dense layers. This was not immediatelly tackled, but there is experimentation with additional generalization methods later on.

In [12]:
from tensorflow.keras import layers, models
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Dropout
from tensorflow.keras.layers import Flatten, Dense

In [19]:
def compileModel (model):
    model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse")] #tracking rmse, as suggested by the competition
        )

    return model

In [20]:
def buildVanillaAlexnet ():
    model = Sequential()
    
    model.add(Input((227, 227, 3)))
    
    model.add(Conv2D(96, 11, 4, activation='relu'))
    model.add(MaxPooling2D(3, 2))

    model.add(Conv2D(256, 5, 1, activation='relu'))
    model.add(MaxPooling2D(3, 2))

    model.add(Conv2D(384, 3, 1, activation='relu'))
    model.add(Conv2D(384, 3, 1, activation='relu'))
    model.add(Conv2D(256, 3, 1, activation='relu'))
    model.add(MaxPooling2D(3, 2))

    model.add(Flatten())

    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(37, activation='sigmoid'))

    model.summary()

    return model

In [21]:
anet = buildVanillaAlexnet()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)                   │ (None, 55, 55, 96)          │          34,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 27, 27, 96)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 23, 23, 256)         │         614,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 11, 11, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_12 (Conv2D)                   │ (None, 9, 9, 384)           │         885,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 7, 7, 384)           │       1,327,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 5, 5, 256)           │         884,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 2, 2, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 4096)                │       4,198,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 4096)                │      16,781,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 37)                  │         151,589 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,878,501 (94.90 MB)

 Trainable params: 24,878,501 (94.90 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
anet = compileModel(anet)

#### 5. Training the vanilla model

In [24]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(  #saving the best model
    "best_model.weights.h5",
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True
)

In [ ]:
history = model.fit(
    tf_train_ds,
    validation_data=tf_val_ds,
    epochs=1,
    callbacks=[checkpoint_cb]
)